##### Copyright 2023 Google LLC

In [2]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

### Install & import

In [9]:
!pip install -U -q google-generativeai

In [10]:
# Install the client library and import necessary modules.
import google.generativeai as genai

import base64
import copy
import hashlib
import io
import json
import mimetypes
import pathlib
import pprint
import requests


import PIL.Image
import IPython.display
from IPython.display import Markdown

## Set the API key

Add your API_KEY to the secrets manager in the left panel "🔑".

In [15]:
from google.colab import userdata

GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")

SecretNotFoundError: Secret GOOGLE_API_KEY does not exist.

In [ ]:
# Configure the client library by providing your API key.
genai.configure(api_key=GOOGLE_API_KEY)

## Parse the arguments

In [ ]:
model = 'gemini-1.5-flash' # @param {isTemplate: true}
contents_b64 = 'W3sicm9sZSI6InVzZXIiLCJwYXJ0cyI6W3sidGV4dCI6IndoYXQgaXMgZWFydGggIn1dfSx7InJvbGUiOiJtb2RlbCIsInBhcnRzIjpbeyJ0ZXh0IjoiRWFydGggaXMgdGhlIHRoaXJkIHBsYW5ldCBmcm9tIHRoZSBTdW4gYW5kIHRoZSBvbmx5IGtub3duIGNlbGVzdGlhbCBib2R5IHRvIGhhcmJvciBsaWZlLiBJdCdzIGEgZHluYW1pYyBhbmQgY29tcGxleCBzeXN0ZW0sIHdpdGggbWFueSBpbnRlcmNvbm5lY3RlZCBwYXJ0cyB0aGF0IG1ha2UgaXQgdW5pcXVlLiBIZXJlJ3MgYSBicmVha2Rvd246XFxuXFxuKipQaHlzaWNhbCBjaGFyYWN0ZXJpc3RpY3M6KipcXG5cXG4qICoqU2l6ZSBhbmQgc2hhcGU6KiogRWFydGggaXMgYW4gb2JsYXRlIHNwaGVyb2lkLCBtZWFuaW5nIGl0J3Mgc2xpZ2h0bHkgZmxhdHRlbmVkIGF0IHRoZSBwb2xlcyBhbmQgYnVsZ2luZyBhdCB0aGUgZXF1YXRvci4gSXQgaGFzIGEgZGlhbWV0ZXIgb2YgYWJvdXQgMTIsNzQyIGtpbG9tZXRlcnMgKDcsOTI2IG1pbGVzKS5cXG4qICoqQ29tcG9zaXRpb246KiogRWFydGggaXMgcHJpbWFyaWx5IGNvbXBvc2VkIG9mIGlyb24sIG94eWdlbiwgc2lsaWNvbiwgbWFnbmVzaXVtLCBzdWxmdXIsIG5pY2tlbCwgY2FsY2l1bSwgYW5kIGFsdW1pbnVtLiAgXFxuKiAqKlN0cnVjdHVyZToqKiBUaGUgRWFydGggaXMgY29tcG9zZWQgb2Ygc2V2ZXJhbCBsYXllcnM6XFxuICAgICogKipDcnVzdDoqKiBUaGUgb3V0ZXJtb3N0IGxheWVyLCB0aGluIGFuZCByaWdpZCwgY29tcG9zZWQgb2YgdmFyaW91cyByb2Nrcy5cXG4gICAgKiAqKk1hbnRsZToqKiBBIHRoaWNrLCBzZW1pLXNvbGlkIGxheWVyIG1hZGUgb2YgaG90IHJvY2suXFxuICAgICogKipPdXRlciBjb3JlOioqIEEgbGlxdWlkIGxheWVyIHByaW1hcmlseSBjb21wb3NlZCBvZiBpcm9uIGFuZCBuaWNrZWwuXFxuICAgICogKipJbm5lciBjb3JlOioqIEEgc29saWQgYmFsbCBvZiBtb3N0bHkgaXJvbiBhbmQgbmlja2VsLCBpbmNyZWRpYmx5IGhvdCBhbmQgZGVuc2UuXFxuKiAqKkF0bW9zcGhlcmU6KiogIEEgbGF5ZXIgb2YgZ2FzZXMgc3Vycm91bmRpbmcgdGhlIEVhcnRoLCBtYWlubHkgY29tcG9zZWQgb2Ygbml0cm9nZW4gYW5kIG94eWdlbi4gSXQgaGVscHMgcmVndWxhdGUgdGVtcGVyYXR1cmUsIHByb3RlY3RzIHVzIGZyb20gaGFybWZ1bCByYWRpYXRpb24sIGFuZCBlbmFibGVzIHdlYXRoZXIgYW5kIGNsaW1hdGUuXFxuKiAqKkh5ZHJvc3BoZXJlOioqIEFsbCB0aGUgd2F0ZXIgb24gRWFydGgsIGluY2x1ZGluZyBvY2VhbnMsIGxha2VzLCByaXZlcnMsIGljZSwgYW5kIGdyb3VuZHdhdGVyLlxcblxcbioqTGlmZToqKlxcblxcbiogRWFydGggaXMgdGhlIG9ubHkga25vd24gcGxhbmV0IGluIG91ciBzb2xhciBzeXN0ZW0gdG8gc3VwcG9ydCBsaWZlLlxcbiogSXQncyB0ZWVtaW5nIHdpdGggZGl2ZXJzZSBlY29zeXN0ZW1zLCBmcm9tIHRoZSBkZWVwZXN0IG9jZWFucyB0byB0aGUgaGlnaGVzdCBtb3VudGFpbnMuXFxuKiBMaWZlIG9uIEVhcnRoIGlzIGluY3JlZGlibHkgZGl2ZXJzZSwgd2l0aCBtaWxsaW9ucyBvZiBzcGVjaWVzLCBpbmNsdWRpbmcgcGxhbnRzLCBhbmltYWxzLCBmdW5naSwgYmFjdGVyaWEsIGFuZCB2aXJ1c2VzLlxcblxcbioqT3RoZXIgZmVhdHVyZXM6KipcXG5cXG4qICoqUGxhdGUgdGVjdG9uaWNzOioqIFRoZSBFYXJ0aCdzIGNydXN0IGlzIGRpdmlkZWQgaW50byBwbGF0ZXMgdGhhdCBtb3ZlIGFuZCBpbnRlcmFjdCwgY2F1c2luZyBlYXJ0aHF1YWtlcywgdm9sY2FuaWMgZXJ1cHRpb25zLCBhbmQgbW91bnRhaW4gZm9ybWF0aW9uLlxcbiogKipNYWduZXRpYyBmaWVsZDoqKiAgVGhlIEVhcnRoJ3MgY29yZSBnZW5lcmF0ZXMgYSBtYWduZXRpYyBmaWVsZCB0aGF0IHByb3RlY3RzIHVzIGZyb20gaGFybWZ1bCBzb2xhciByYWRpYXRpb24uXFxuKiAqKk1vb246KiogRWFydGggaGFzIGEgc2luZ2xlIG5hdHVyYWwgc2F0ZWxsaXRlLCB0aGUgTW9vbiwgd2hpY2ggaW5mbHVlbmNlcyB0aWRlcyBhbmQgc3RhYmlsaXplcyBFYXJ0aCdzIHJvdGF0aW9uLlxcblxcbioqU2lnbmlmaWNhbmNlOioqXFxuXFxuKiBFYXJ0aCBpcyBvdXIgaG9tZSwgcHJvdmlkaW5nIHVzIHdpdGggdGhlIHJlc291cmNlcyBhbmQgZW52aXJvbm1lbnQgd2UgbmVlZCB0byBzdXJ2aXZlLlxcbiogSXQncyBhIGZhc2NpbmF0aW5nIGFuZCBjb21wbGV4IHBsYW5ldCwgY29uc3RhbnRseSBldm9sdmluZyBhbmQgY2hhbmdpbmcuXFxuKiBVbmRlcnN0YW5kaW5nIEVhcnRoIGlzIGNydWNpYWwgZm9yIHNvbHZpbmcgbWFueSBjaGFsbGVuZ2VzIGZhY2luZyBodW1hbml0eSwgaW5jbHVkaW5nIGNsaW1hdGUgY2hhbmdlLCByZXNvdXJjZSBkZXBsZXRpb24sIGFuZCBlbnZpcm9ubWVudGFsIGRlZ3JhZGF0aW9uLlxcblxcbioqRXhwbG9yaW5nIEVhcnRoOioqXFxuXFxuKiBXZSBoYXZlIGJlZW4gZXhwbG9yaW5nIEVhcnRoIGZvciBjZW50dXJpZXMsIG1hcHBpbmcgaXRzIHN1cmZhY2UsIHN0dWR5aW5nIGl0cyBnZW9sb2d5LCBhbmQgcmVzZWFyY2hpbmcgaXRzIGF0bW9zcGhlcmUgYW5kIG9jZWFucy5cXG4qIE1vZGVybiB0ZWNobm9sb2d5LCBpbmNsdWRpbmcgc2F0ZWxsaXRlcywgc3BhY2VjcmFmdCwgYW5kIGFkdmFuY2VkIHNjaWVudGlmaWMgaW5zdHJ1bWVudHMsIGFsbG93cyB1cyB0byBzdHVkeSBFYXJ0aCBpbiB1bnByZWNlZGVudGVkIGRldGFpbC5cXG5cXG5VbmRlcnN0YW5kaW5nIEVhcnRoIGlzIGEgY29udGludW91cyBqb3VybmV5IG9mIGRpc2NvdmVyeS4gV2l0aCBldmVyeSBuZXcgb2JzZXJ2YXRpb24gYW5kIHJlc2VhcmNoLCB3ZSBnYWluIGRlZXBlciBpbnNpZ2h0cyBpbnRvIHRoZSBwbGFuZXQgd2UgY2FsbCBob21lLiBcXG4ifV19LHsicm9sZSI6InVzZXIiLCJwYXJ0cyI6W3siZmlsZV9kYXRhIjp7Im1pbWVfdHlwZSI6ImltYWdlL2pwZWciLCJ1cmwiOiJodHRwczovL3d3dy5nc3RhdGljLmNvbS9haXN0dWRpby9zYW1wbGUtaW1hZ2VzL2FuaW1hbDMuanBnIn19LHsidGV4dCI6InRoaXMgcGljdHVyZSBpcyBvZiB3aGF0XFxuIn1dfSx7InJvbGUiOiJtb2RlbCIsInBhcnRzIjpbeyJ0ZXh0IjoiVGhpcyBwaWN0dXJlIGlzIG9mIGEgSGF2YW5lc2UgcHVwcHkuIFxcbiJ9XX1d' # @param {isTemplate: true}
generation_config_b64 = 'eyJ0ZW1wZXJhdHVyZSI6MSwidG9wX3AiOjAuOTUsInRvcF9rIjo2NCwibWF4X291dHB1dF90b2tlbnMiOjgxOTJ9' # @param {isTemplate: true}
safety_settings_b64 = "e30="  # @param {isTemplate: true}

gais_contents = json.loads(base64.b64decode(contents_b64))

generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))

stream = False

In [ ]:
gais_contents

In [ ]:
generation_config

In [ ]:
safety_settings

## [optional] Show the conversation

This section displays the conversation received from Google AI Studio.

In [ ]:
# @title `show_file` function
drive = None
def show_file(file_data):
    mime_type = file_data["mime_type"]

    if drive_id := file_data.get("drive_id", None):
        if drive is None:
          from google.colab import drive
          drive.mount("/gdrive")
        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        name = path
        # data = path.read_bytes()
        kwargs = {"filename": path}
    elif url := file_data.get("url", None):
        name = url
        kwargs = {"url": url}
        # response = requests.get(url)
        # data = response.content
    elif data := file_data.get("inline_data", None):
        name = None
        kwargs = {"data": data}
    elif name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files to "
                'Colab using the file manager ("📁 Files"in the left toolbar)'
            )
    else:
        raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")

        print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")
        return

    format = mimetypes.guess_extension(mime_type).strip(".")
    if mime_type.startswith("image/"):
        image = IPython.display.Image(**kwargs, width=256)
        IPython.display.display(image)
        print()
        return

    if mime_type.startswith("audio/"):
        if len(data) < 2**12:
            audio = IPython.display.Audio(**kwargs)
            IPython.display.display(audio)
            print()
            return

    if mime_type.startswith("video/"):
        if len(data) < 2**12:
            audio = IPython.display.Video(**kwargs, mimetype=mime_type)
            IPython.display.display(audio)
            print()
            return

    print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")

In [ ]:
for content in gais_contents:
    if role := content.get("role", None):
        print("Role:", role, "\n")

    for n, part in enumerate(content["parts"]):
        if text := part.get("text", None):
            print(text, "\n")

        elif file_data := part.get("file_data", None):
            show_file(file_data)

    print("-" * 80, "\n")

## Convert & upload files

For each file, Google AI Studio either sent:
- a Google Drive IDs
- a URL, or
- the raw bytes (`inline_data`).

The API itself onlty understands two ways of sending files:

- `inline_data` - where the bytes are placed inline in the request.
- `file_data` - where the file is uploaded to the Files API, and you pass a reference to that file.

This section goes through the `contents` from Google AI Studio, and uploads the file data, to the Files API, so Gemini can access it:

In [ ]:
# @title `upload_file` function

tempfiles = pathlib.Path(f"tempfiles")
tempfiles.mkdir(parents=True, exist_ok=True)

drive = None
def upload_file_data(file_data):
    mime_type = file_data["mime_type"]
    if drive_id := file_data.pop("drive_id", None):
        if drive is None:
          from google.colab import drive
          drive.mount("/gdrive")

        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        print("Uploading:", str(path))
        file_info = genai.upload_file(path=path, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if url := file_data.pop("url", None):
        response = requests.get(url)
        data = response.content
        name = url.split("/")[-1]
        hash = hashlib.sha256(data).hexdigest()
        path = tempfiles / hash
        path.write_bytes(data)
        print("Uploading:", url)
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files "
                'to Colab using the file manager ("📁 Files"in the left '
                "toolbar)"
            )
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if "inline_data" in file_data:
        return

    raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")

In [ ]:
contents = copy.deepcopy(gais_contents)

for content in contents:
    for n, part in enumerate(content["parts"]):
        if file_data := part.get("file_data", None):
            upload_file_data(file_data)

Here is the coverted `Content`s:

In [ ]:
contents

## Call `generate_content`

In [ ]:
# Call the model and print the response.
gemini = genai.GenerativeModel(model_name=model)

response = gemini.generate_content(
    contents,
    generation_config=generation_config,
    safety_settings=safety_settings,
    stream=False,
)

In [ ]:
if generation_config.get("candidate_count", 1) == 1:
    display(Markdown(response.text))

In [ ]:
response.candidates

## Or Create a chat

In [ ]:
gemini = genai.GenerativeModel(
    model_name=model,
    generation_config=generation_config,
    safety_settings=safety_settings,
)

A `ChatSession`, should always end with the model's turn.

So before creating the `chat` check whos turn was last.

If the user was last, attach all but the last content as the `history` and send the last content with `send_message` to get the model's response.

If the model was last, put the whole contents list in the history.

In [ ]:
model_turn = contents[-1].get("role", None) == "user"

if model_turn:
    chat = gemini.start_chat(history=contents[:-1])

    response = chat.send_message(contents[-1])

    if generation_config.get("candidate_count", 1) == 1:
        display(Markdown(response.text))
else:
    chat = gemini.start_chat(history=contents)

In [ ]:
if model_turn:
    response.candidates

After that use `send_message` to continue the conversation.

In [ ]:
# response = chat.send_message("Interesting, tell me more.")
# display(Markdown(response.text))